# Onsets

In [1]:
import os, glob
import pandas as pd
import numpy 
import ipywidgets as widgets

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)



## Set and load data

In [5]:
onsets=glob.glob('/Users/nikkibytes/Documents/local_dev/bbx/output_onsets/*txt')
logfiles=glob.glob('/Users/nikkibytes/Documents/local_dev/bbx/log_files/bbx_*')

onsets.sort()
logfiles.sort()

In [6]:
sub_ids=[x.split("/")[-1].split("_")[0] for x in onsets]
sub_ids=list(set(sub_ids))
sub_ids.sort()

In [7]:
print(onsets[2], "\n")
#for line in open(onsets[2], 'r'):
 #   print(line)

/Users/nikkibytes/Documents/local_dev/bbx/output_onsets/sub-001_ses-1_task-SSBcue_run-3.txt 



## Inspect Code 

In [8]:
# Helper functions
def set_logfile():
    return logdata;

def set_onset_file():
    return onsetdata;

def group_ev_data():
    return grouped_data;

In [9]:
data_dict={}

In [10]:
# onset widget viewer

## Original Code

In [11]:
handles = []

## Change this path to match where you have the onset files
basepath = '/Users/nikkibytes/Documents/local_dev/bbx/log_files'
os.chdir(basepath)

ignore = ['DATA 	Keypress: o', 'Level post injecting via pump at address']

# files = [file for file in os.listdir(".") if (file.lower().endswith('.log'))]
# files.sort(key=os.path.getmtime)

# get the global info about the run.

logfiles=glob.glob(os.path.join(basepath, '*.log'))
logfiles.sort()
bbx_ids=[x.split("/")[-1].split("_")[0:4] for x in logfiles if "pre" in x or "post" in x]
bbx_ids=['_'.join(x) for x in bbx_ids]

print(bbx_ids[:3])


['bbx_001_run01_post', 'bbx_001_run01_pre', 'bbx_001_run02_post']


In [15]:

def view_onsets(file, data_dict):
    # This will change dependign on the path -- rememeber to count starting at 0 before the first '/'
    sub = file.split('/')[-1].split('_')[1]
    run = file.split('/')[-1].split('_')[2]
    wave = file.split('/')[-1].split('_')[3]
    f_id="%s_%s_%s"%(sub,run,wave)
    # setup dict
    if f_id not in data_dict:
        data_dict[f_id]={}

    print('[INFO] %s %s %s'%(sub, run, wave))
    print('[INFO] filename: ', file)
    logf_df=pd.read_csv(logfiles[0], sep="\t", header=None)
    #print("[INFO] logfile dataframe: ")

    logf_df.columns =["time", "data", "keypress"]
    print("\n[INFO] logfile: \n",logf_df)
    #display(logf_df.head())

    data_dict[f_id]['logfile_df']=logf_df

    #   open the script and read in log data
    with open(file, 'r') as infile:
        ssb_cue_onset = []
        ssb_taste_onset = []
        usb_cue_onset = []
        usb_taste_onset = []
        h2o_cue_onset = []
        h2o_taste_onset = []
        neu_onset = []
        start_time = None
        for x in infile.readlines():
            #            if x.find('Keypress: q'):
            #                continue

            if not x.find(ignore[0]) > -1 or x.find(ignore[1]) > -1:

                l_s = x.strip().split()

                if x.find('Level start key press') > -1:  # find the start
                    l_s = x.strip().split()
                    start_time = float(l_s[0])
                    print('[INFO] start time: ', start_time)

                # if x.find('at time= ')>1:
                # l_s=x.strip().split()
                # RT.append(l_s[5])

                # SSB CUE AND TASTE
                # The SSBs are  either "C0" or "SL" -- no one will receive both, so we can repeat in the parser :)
                if x.find('image=CO.jpg') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] ssbcue:', ','.join(l_s))
                    ssb_cue_onset.append(float(l_s[0]))

                if x.find('image=SL.jpg') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] ssbcue:',l_s)
                    ssb_cue_onset.append(float(l_s[0]))

                # NOTE: we use the post injecting time because it is only listed once
                if x.find('Level post injecting via pump at address 1') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] ssbtaste:', ','.join(l_s))
                    ssb_taste_onset.append(float(l_s[0]))

                # USB CUE AND TASTE
                # Just like the SSBs are we can repeat the USB names in the parser
                if x.find('image=UCO.jpg') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] usbbcue:', ','.join(l_s))
                    usb_cue_onset.append(float(l_s[0]))

                if x.find('image=USL.jpg') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] usbbcue:', ','.join(l_s))
                    usb_cue_onset.append(float(l_s[0]))

                if x.find('Level post injecting via pump at address 2') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] usbbtaste:', ','.join(l_s))
                    usb_taste_onset.append(float(l_s[0]))

                # WATER CUE AND TASTE
                if x.find('image=water.jpg') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] water cue:', ','.join(l_s))
                    h2o_cue_onset.append(float(l_s[0]))

                if x.find('Level post injecting via pump at address 0') > -1:
                    l_s = x.strip().split()
                    #print('[INFO] water taste:', ','.join(l_s))
                    h2o_taste_onset.append(float(l_s[0]))

                # rinse
                if x.find('Level RINSE 	25') > -1:
                    neu_onset.append(l_s[0])
                    #print('[INFO] rinse:', ','.join(l_s))


        print('[INFO] SSBcue: ',ssb_cue_onset)
        print('[INFO] SSBtaste: ', ssb_taste_onset)



        # ssbcue 
        #######################
        # compare the two dataframes 
        # they query the same logfile two different ways 

        # set dataframe with pulled timepoints 
        ssbcue_onsets = pd.DataFrame({ 
                "onset": (numpy.asarray(ssb_cue_onset, dtype=float)) - start_time
                })

        # add stim time and mod column
        ssbcue_onsets["stim"]=1
        ssbcue_onsets["mod"] = 1
        # display the  ev onsets 
        print("\n[INFO] SSBCUE onsets pulled with traversing file method: \n", ssbcue_onsets)

        # query the logfile dataframe and grab the timepoints too and make a dataframe
        ssbcue_query=logf_df[(logf_df["data"].str.contains("image=CO.jpg")) | (logf_df["data"].str.contains("image=SL.jpg"))]
        #ssbcue_query.time -= start_time
        data_dict[f_id]['ssbcue_query'] = ssbcue_query  
        print("\n[INFO] SSBCUE onsets pulled : \n", ssbcue_query)

        print("------------------------------------------------------------------------\n")



        # ssbtaste 
        ###################

        ssbtaste_onsets = pd.DataFrame({ 
                "onset": (numpy.asarray(ssb_taste_onset, dtype=float)) - start_time
                })
        # add stim time and mod column
        ssbtaste_onsets["stim"]=6
        ssbtaste_onsets["mod"] = 1
        # display the  ev onsets 
        print("[INFO] SSB TASTE onsets pulled out from lines file \n", ssbtaste_onsets)

        # query the logfile dataframe and grab the timepoints too and make a dataframe
        ssbtaste_query=logf_df[logf_df["data"].str.contains("Level post injecting via pump at address 1")]
        ssbtaste_query.time = ssbtaste_query.time - start_time
        data_dict[f_id]['ssbtaste_query'] = ssbtaste_query
        print("\n[INFO] SSB TASTE queried in logfile with pandas method: \n", ssbtaste_query)

        print("------------------------------------------------------------------------\n")



        # usbcue
        #################
        usbcue_onsets = pd.DataFrame({ 
                "onset": (numpy.asarray(usb_cue_onset, dtype=float)) - start_time
                })

        # add stim time and mod column
        usbcue_onsets["stim"]=1
        usbcue_onsets["mod"] = 1
        # display the  ev onsets 
        print("\n[INFO] USB CUE onsets pulled out from file \n", usbcue_onsets)

        # query the logfile dataframe and grab the timepoints too and make a dataframe
        usbcue_query=logf_df[(logf_df["data"].str.contains("image=UCO.jpg")) | (logf_df["data"].str.contains("image=USL.jpg"))]
        usbcue_query.time = usbcue_query.time - start_time
        data_dict[f_id]['usbcue_query'] = usbcue_query  
        print("\n[INFO] USB CUE queried in logfile: \n", usbcue_query)

        print("------------------------------------------------------------------------\n")



        # usbtaste 
        ################
        usbtaste_onsets = pd.DataFrame({ 
                "onset": (numpy.asarray(usb_taste_onset, dtype=float)) - start_time
                })
        # add stim time and mod column
        usbtaste_onsets["stim"]=6
        usbtaste_onsets["mod"] = 1
        # display the  ev onsets 
        print("\n[INFO] USB TASTE onsets pulled out from lines file \n", usbtaste_onsets)

        # query the logfile dataframe and grab the timepoints too and make a dataframe
        usbtaste_query=logf_df[logf_df["data"].str.contains("Level post injecting via pump at address 2")]
        usbtaste_query.time = usbtaste_query.time - start_time
        data_dict[f_id]['usbtaste_query'] = usbtaste_query

        print("\n[INFO] USB TASTE queried in logfile with pandas method: \n", usbtaste_query)

        print("------------------------------------------------------------------------\n")




        # h2O cue
        ###################
        h2Ocue_onsets = pd.DataFrame({ 
                "onset": (numpy.asarray(h2o_cue_onset, dtype=float)) - start_time
                })

        # add stim time and mod column
        h2Ocue_onsets["stim"]=1
        h2Ocue_onsets["mod"] = 1
        # display the  ev onsets 
        print("\n[INFO] H2O CUE onsets pulled out from file \n", h2Ocue_onsets)

        # query the logfile dataframe and grab the timepoints too and make a dataframe
        h2Ocue_query=logf_df[(logf_df["data"].str.contains("image=water.jpg"))]
        h2Ocue_query.time = h2Ocue_query.time - start_time
        data_dict[f_id]['h2Ocue_query'] = h2Ocue_query  
        print("\n[INFO] H2O CUE queried in logfile: \n", h2Ocue_query)

        print("------------------------------------------------------------------------\n")



        # h2Otaste
        ################
        h2Otaste_onsets = pd.DataFrame({ 
                "onset": (numpy.asarray(h2o_taste_onset, dtype=float)) - start_time
                })
        # add stim time and mod column
        h2Otaste_onsets["stim"]=6
        h2Otaste_onsets["mod"] = 1
        # display the  ev onsets 
        print("\n[INFO] H2O TASTE onsets pulled out from lines file \n", h2Otaste_onsets)

        # query the logfile dataframe and grab the timepoints too and make a dataframe
        h2Otaste_query=logf_df[logf_df["data"].str.contains("Level post injecting via pump at address 0")]
        h2Otaste_query.time = h2Otaste_query.time - start_time
        data_dict[f_id]['h2Otaste_query'] = h2Otaste_query

        print("\n[INFO] H2O TASTE queried in logfile with pandas method: \n", h2Otaste_query)

        print("------------------------------------------------------------------------\n")

        #print("[INFO] rinse onsets: \n", display(neu_onsets))

        # rinse
        ################
        rinse_onsets = pd.DataFrame({ 
                "onset": (numpy.asarray(neu_onset, dtype=float)) - start_time
                })
        # add stim time and mod column
        rinse_onsets["stim"]=3
        rinse_onsets["mod"] = 1
        # display the  ev onsets 
        print("\n[INFO] RINSE onsets pulled out from lines file \n", rinse_onsets)

        # query the logfile dataframe and grab the timepoints too and make a dataframe
        rinse_query=logf_df[logf_df["data"].str.contains('Level RINSE')]
        rinse_query.time -= start_time
        data_dict[f_id]['rinse_query'] = rinse_query

        print("\n[INFO] RINSE queried in logfile with pandas method: \n", rinse_query)

        print("\n------------------------------------------------------------------------\n")

        return data_dict;

In [16]:
# make widget for onset viewer 

def display_logs(subject=bbx_ids[0]):
    handles = []
    data_dict={}
    ## Change this path to match where you have the onset files
    basepath = '/Users/nikkibytes/Documents/local_dev/bbx/log_files'
    os.chdir(basepath)

    ignore = ['DATA 	Keypress: o', 'Level post injecting via pump at address']

    # files = [file for file in os.listdir(".") if (file.lower().endswith('.log'))]
    # files.sort(key=os.path.getmtime)

    # get the global info about the run.

    logfiles=glob.glob(os.path.join(basepath, '%s*.log'%subject))
    logfiles.sort()

    if len(logfiles) > 2:
        print("[INFO] multiple logfiles found for subjects session run: \n%s %s"%(logfiles))
        pass
    else:
        
        input_file=logfiles[0]
        data_dict=view_onsets(input_file, data_dict)

        
            

In [17]:
# run widget that calls help functions above
w_log = widgets.Select(options=bbx_ids)
widgets.interactive(display_logs, subject=w_log)

interactive(children=(Select(description='subject', options=('bbx_001_run01_post', 'bbx_001_run01_pre', 'bbx_0…

In [ ]:
def custom_bbx_logfile_display(output, query, ev):
    

In [ ]:
# run widget that calls help functions above
logfile_widget = widgets.Select(options=bbx_ids)
widgets.interactive(display_logs, subject=w_log)

In [ ]:
# make bbx dataframe

In [48]:
EVS=["SSBcue", "USBcue", "SSBtaste", "USBtaste", "H2Ocue", "H2Otaste", "rinse"]

# function to display ev files
def display_ev(unique_id, unique_ev):
    print('\n[INFO] id:%s ev:%s'%(unique_id, unique_ev))
    
    
    if unique_ev == "SSBcue":
        
    
# function to display ev files
#def set_ev(_):
    #if unique_ev.value == "SSBcue":
        


unique_id = widgets.Dropdown(
        options=bbx_ids,
        value=bbx_ids[0],
        description='unique id'
    )


#unique_id.observe(set_ev)
#headers_ev.observe()

unique_ev = widgets.Dropdown(
        options=evs,
        value=EVS[0],
        description='select ev to view'
    )

unique_id.set_title='unique_id'

widgets.interact(display_ev, unique_id=unique_id, unique_ev=unique_ev)

interactive(children=(Dropdown(description='unique id', options=('bbx_001_run01_post', 'bbx_001_run01_pre', 'b…

<function __main__.display_ev(unique_id, unique_ev)>

In [24]:
def open_logfile(unique_id, data_dict):
    handles = []
    
    ## Change this path to match where you have the onset files
    basepath = '/Users/nikkibytes/Documents/local_dev/bbx/log_files'
    os.chdir(basepath)

    ignore = ['DATA 	Keypress: o', 'Level post injecting via pump at address']

    # files = [file for file in os.listdir(".") if (file.lower().endswith('.log'))]
    # files.sort(key=os.path.getmtime)

    # get the global info about the run.

    logfiles=glob.glob(os.path.join(basepath, '%s*.log'%unique_id))
    logfiles.sort()
    
    
    for file in logfiles:

        if "pre" not in file and "post" not in file:
            pass
        else:



            # This will change dependign on the path -- rememeber to count starting at 0 before the first '/'
            sub = file.split('/')[-1].split('_')[1]
            run = file.split('/')[-1].split('_')[2]
            wave = file.split('/')[-1].split('_')[3]
            f_id="%s_%s_%s"%(sub,run,wave)
            # setup dict
            if f_id not in data_dict:
                data_dict[f_id]={}

            print('[INFO] %s %s %s'%(sub, run, wave))
            print('[INFO] log file: ', file)
            logf_df=pd.read_csv(file, sep="\t", header=None)
            #print("[INFO] logfile dataframe: ")

            logf_df.columns =["time", "data", "keypress"]
            #display(logf_df.head())

            data_dict[f_id]['logfile_df']=logf_df

            #   open the script and read in log data
            with open(file, 'r') as infile:
                ssb_cue_onset = []
                ssb_taste_onset = []
                usb_cue_onset = []
                usb_taste_onset = []
                h2o_cue_onset = []
                h2o_taste_onset = []
                neu_onset = []
                start_time = None
                for x in infile.readlines():
                    #            if x.find('Keypress: q'):
                    #                continue

                    if not x.find(ignore[0]) > -1 or x.find(ignore[1]) > -1:

                        l_s = x.strip().split()

                        if x.find('Level start key press') > -1:  # find the start
                            l_s = x.strip().split()
                            start_time = float(l_s[0])
                            print('[INFO] start time: ', start_time)

                        # if x.find('at time= ')>1:
                        # l_s=x.strip().split()
                        # RT.append(l_s[5])

                        # SSB CUE AND TASTE
                        # The SSBs are  either "C0" or "SL" -- no one will receive both, so we can repeat in the parser :)
                        if x.find('image=CO.jpg') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] ssbcue:', ','.join(l_s))
                            ssb_cue_onset.append(float(l_s[0]))

                        if x.find('image=SL.jpg') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] ssbcue:',l_s)
                            ssb_cue_onset.append(float(l_s[0]))

                        # NOTE: we use the post injecting time because it is only listed once
                        if x.find('Level post injecting via pump at address 1') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] ssbtaste:', ','.join(l_s))
                            ssb_taste_onset.append(float(l_s[0]))

                        # USB CUE AND TASTE
                        # Just like the SSBs are we can repeat the USB names in the parser
                        if x.find('image=UCO.jpg') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] usbbcue:', ','.join(l_s))
                            usb_cue_onset.append(float(l_s[0]))

                        if x.find('image=USL.jpg') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] usbbcue:', ','.join(l_s))
                            usb_cue_onset.append(float(l_s[0]))

                        if x.find('Level post injecting via pump at address 2') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] usbbtaste:', ','.join(l_s))
                            usb_taste_onset.append(float(l_s[0]))

                        # WATER CUE AND TASTE
                        if x.find('image=water.jpg') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] water cue:', ','.join(l_s))
                            h2o_cue_onset.append(float(l_s[0]))

                        if x.find('Level post injecting via pump at address 0') > -1:
                            l_s = x.strip().split()
                            #print('[INFO] water taste:', ','.join(l_s))
                            h2o_taste_onset.append(float(l_s[0]))

                        # rinse
                        if x.find('Level RINSE 	25') > -1:
                            neu_onset.append(l_s[0])
                            #print('[INFO] rinse:', ','.join(l_s))


                print('[INFO] SSBcue: ',ssb_cue_onset)
                print('[INFO] SSBtaste: ', ssb_taste_onset)



                # ssbcue 
                #######################
                # compare the two dataframes 
                # they query the same logfile two different ways 

                # set dataframe with pulled timepoints 

                ssbcue_onsets = pd.DataFrame({ 
                        "onset": (numpy.asarray(ssb_cue_onset, dtype=float)) - start_time
                        })

                # add stim time and mod column
                ssbcue_onsets["stim"]=1
                ssbcue_onsets["mod"] = 1
                # display the  ev onsets 
                print("\n[INFO] SSBCUE onsets pulled out from file \n", ssbcue_onsets)
                # query the logfile dataframe and grab the timepoints too and make a dataframe
                ssbcue_query=logf_df[(logf_df["data"].str.contains("image=CO.jpg")) | 
                                      (logf_df["data"].str.contains("image=SL.jpg"))]
                ssbcue_query.time -= start_time
                data_dict[f_id]['ssbcue_query'] = ssbcue_query  
                print("\n[INFO] SSB CUE queried in logfile: \n", ssbcue_query)
                print("\n------------------------------------------------------\n")


                # ssbtaste 
                ###################

                ssbtaste_onsets = pd.DataFrame({ 
                        "onset": (numpy.asarray(ssb_taste_onset, dtype=float)) - start_time
                        })
                # add stim time and mod column
                ssbtaste_onsets["stim"]=6
                ssbtaste_onsets["mod"] = 1
                # display the  ev onsets 
                print("[INFO] SSB TASTE onsets pulled out from lines file \n", ssbtaste_onsets)

                # query the logfile dataframe and grab the timepoints too and make a dataframe
                ssbtaste_query=logf_df[logf_df["data"].str.contains("Level post injecting via pump at address 1")]
                ssbtaste_query.time = ssbtaste_query.time - start_time
                data_dict[f_id]['ssbtaste_query'] = ssbtaste_query
                print("\n[INFO] SSB TASTE queried in logfile with pandas method: \n", ssbtaste_query)

                print("\n------------------------------------------------------\n")



                # usbcue
                #################
                usbcue_onsets = pd.DataFrame({ 
                        "onset": (numpy.asarray(usb_cue_onset, dtype=float)) - start_time
                        })

                # add stim time and mod column
                usbcue_onsets["stim"]=1
                usbcue_onsets["mod"] = 1
                # display the  ev onsets 
                print("\n[INFO] USB CUE onsets pulled out from file \n", usbcue_onsets)
                # query the logfile dataframe and grab the timepoints too and make a dataframe
                usbcue_query=logf_df[(logf_df["data"].str.contains("image=UCO.jpg")) | (logf_df["data"].str.contains("image=USL.jpg"))]
                usbcue_query.time = usbcue_query.time - start_time
                data_dict[f_id]['usbcue_query'] = usbcue_query  
                print("\n[INFO] USB CUE queried in logfile: \n", usbcue_query)
                print("\n------------------------------------------------------\n")



                # usbtaste 
                ################
                usbtaste_onsets = pd.DataFrame({ 
                        "onset": (numpy.asarray(usb_taste_onset, dtype=float)) - start_time
                        })
                # add stim time and mod column
                usbtaste_onsets["stim"]=6
                usbtaste_onsets["mod"] = 1
                # display the  ev onsets 
                print("\n[INFO] USB TASTE onsets pulled out from lines file \n", usbtaste_onsets)
                # query the logfile dataframe and grab the timepoints too and make a dataframe
                usbtaste_query=logf_df[logf_df["data"].str.contains("Level post injecting via pump at address 2")]
                usbtaste_query.time -= start_time
                data_dict[f_id]['usbtaste_query'] = usbtaste_query
                print("\n[INFO] USB TASTE queried in logfile with pandas method: \n", usbtaste_query)
                print("\n------------------------------------------------------\n")


                # h2O cue
                ###################
                h2Ocue_onsets = pd.DataFrame({ 
                        "onset": (numpy.asarray(h2o_cue_onset, dtype=float)) - start_time
                        })

                # add stim time and mod column
                h2Ocue_onsets["stim"]=1
                h2Ocue_onsets["mod"] = 1
                # display the  ev onsets 
                print("\n[INFO] H2O CUE onsets pulled out from file \n", h2Ocue_onsets)
                # query the logfile dataframe and grab the timepoints too and make a dataframe
                h2Ocue_query=logf_df[(logf_df["data"].str.contains("image=water.jpg"))]
                h2Ocue_query.time -= start_time
                data_dict[f_id]['h2Ocue_query'] = h2Ocue_query  
                print("\n[INFO] H2O CUE queried in logfile: \n", h2Ocue_query)
                print("\n------------------------------------------------------\n")



                # h2Otaste
                ################
                h2Otaste_onsets = pd.DataFrame({ 
                        "onset": (numpy.asarray(h2o_taste_onset, dtype=float)) - start_time
                        })
                # add stim time and mod column
                h2Otaste_onsets["stim"]=6
                h2Otaste_onsets["mod"] = 1
                # display the  ev onsets 
                print("\n[INFO] H2O TASTE onsets pulled out from lines file \n", h2Otaste_onsets)

                # query the logfile dataframe and grab the timepoints too and make a dataframe
                h2Otaste_query=logf_df[logf_df["data"].str.contains("Level post injecting via pump at address 0")]
                h2Otaste_query.time -= start_time
                data_dict[f_id]['h2Otaste_query'] = h2Otaste_query

                print("\n[INFO] H2O TASTE queried in logfile with pandas method: \n", h2Otaste_query)

                print("\n------------------------------------------------------\n")

                #print("[INFO] rinse onsets: \n", display(neu_onsets))

                # rinse
                ################
                rinse_onsets = pd.DataFrame({ 
                        "onset": (numpy.asarray(neu_onset, dtype=float)) - start_time
                        })
                # add stim time and mod column
                rinse_onsets["stim"]=3
                rinse_onsets["mod"] = 1
                # display the  ev onsets 
                print("\n[INFO] RINSE onsets pulled out from lines file \n", rinse_onsets)

                # query the logfile dataframe and grab the timepoints too and make a dataframe
                rinse_query=logf_df[logf_df["data"].str.contains('Level RINSE')]
                rinse_query.time -= start_time
                data_dict[f_id]['rinse_query'] = rinse_query

                print("\n[INFO] RINSE queried in logfile with pandas method: \n", rinse_query)

                print("\n------------------------------------------------------\n")



In [25]:
data_dict={}
for unique_id in bbx_ids[:2]:
    #print(unique_id)
    open_logfile(unique_id, data_dict)


[INFO] 001 run01 post
[INFO] log file:  /Users/nikkibytes/Documents/local_dev/bbx/log_files/bbx_001_run01_post_2018-04-16-18_35_11_subdata.log
[INFO] start time:  58.7065
[INFO] SSBcue:  [123.6467, 180.6629, 221.6606, 299.652, 365.6539, 400.6363, 434.6566, 472.657]
[INFO] SSBtaste:  [124.4418, 181.4569, 222.4595, 300.4642, 366.4502, 401.4485, 435.453, 473.4535]

[INFO] SSBCUE onsets pulled out from file 
       onset  stim  mod
0   64.9402     1    1
1  121.9564     1    1
2  162.9541     1    1
3  240.9455     1    1
4  306.9474     1    1
5  341.9298     1    1
6  375.9501     1    1
7  413.9505     1    1

[INFO] SSB CUE queried in logfile: 
          time                 data keypress
60    64.9402  Level image=CO.jpg        25
113  121.9564  Level image=CO.jpg        25
152  162.9541  Level image=CO.jpg        25
225  240.9455  Level image=CO.jpg        25
293  306.9474  Level image=CO.jpg        25
328  341.9298  Level image=CO.jpg        25
363  375.9501  Level image=CO.jpg     

## Compare logfiles with generated onset files

## Notes  

Steps in data dictionary(referencing rules from parser file)  

* Find start key   
`Level start key press`  

* SSBcue and SSBtaste  
For cue:  
The SSBs are  either "CO" or "SL" -- no one will receive both, so we can repeat in the parser   
`Level image=CO.jpg` and `Level image=SL.jpg`     
For Taste:  
    **NOTE:** we use the post injecting time because it is only listed once   
`Level post injecting via pump at address 1`  
* USBcue AND SSBtaste    
Just like the SSBs are we can repeat the USB names in the parser  
For cue:  
`image=UCO.jpg` and `image=USL.jpg`   
For Taste:  
`'Level post injecting via pump at address 2'`  
* H2Ocue AND H2Otaste  
For cue:  
`image=water.jpg`
For taste:  
`Level post injecting via pump at address 0`   

* Rinse  
`Level RINSE 	25`


* Minus start time from the onset time   

* Stim/mod values  
{ssbcue: (1,1), ssbtaste: (6,1), usbcue: (1,1), usbtaste: (6,1), h2Ocue: (1,1), h2Otaste: (6,1), rinse: (3,1)}